In [1]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)
from carst import SingleRaster, RasterVelos, ConfParams
from carst.libft import ampcor_task, writeout_ampcor_task
from carst.libxyz import ZArray, DuoZArray, AmpcoroffFile, points_in_polygon

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


1. Firstly let's start with rwo pixel offset.

In [ ]:
ampoff = AmpcoroffFile('/home/whyj/Projects/Github/CARST/examples/featuretrack/Demo_Data/20180401-20180417_ampoff.p')
ampoff.Load()
ampoff.SetIni(ini)
ampoff.FillwithNAN()
ampoff.Ampcoroff2Velo(velo_or_pixel='pixel')

shp = ini.velocorrection['bedrock']
prefix = ini.rawoutput['label_geotiff']
velo = RasterVelos(vx=SingleRaster(prefix + '_vx.tif'),
                   vy=SingleRaster(prefix + '_vy.tif'),
                   snr=SingleRaster(prefix + '_snr.tif'),
                   mag=SingleRaster(prefix + '_mag.tif'),
                   errx=SingleRaster(prefix + '_errx.tif'),
                   erry=SingleRaster(prefix + '_erry.tif'))

idx = points_in_polygon(ampoff.data[:, [0,2]], shp)

# SNR constraint
snr_threshold = ampoff.snr[:, 2] >= ini.noiseremoval['snr']
idx = np.logical_and(idx, snr_threshold)


vxraw_bdval = ZArray(ampoff.velo_x[idx, 2])
vyraw_bdval = ZArray(ampoff.velo_y[idx, 2])
vxyraw_bdval = DuoZArray(z1=vxraw_bdval, z2=vyraw_bdval, ini=ini)
vxyraw_bdval.OutlierDetection2D(thres_sigma=ini.velocorrection['refvelo_outlier_sigma'])
vxyraw_bdval.HistWithOutliers(which='x')
vxyraw_bdval.HistWithOutliers(which='y')